# Baseline experiments

### Импортируем библиотеки

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.decomposition import KernelPCA
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomTreesEmbedding
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import StackingClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.svm import LinearSVC

C:\Users\kyuda\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


### Эксперименты с Random Forest и ансамблями на фичах V1_B5_456

In [2]:
# Загружаем данные
X_train = pd.read_csv("./processed_images_data/X_train_V1_B5_456.csv",  header=None)
y_train = pd.read_csv("./processed_images_data/y_train_V1_B5_456.csv", header=None).values.ravel()
X_test = pd.read_csv("./processed_images_data/X_test_V1_B5_456.csv",  header=None)
y_test = pd.read_csv("./processed_images_data/y_test_V1_B5_456.csv", header=None).values.ravel()

In [10]:
# Baseline RF с дефолтными параметрами
model = RandomForestClassifier(n_jobs=-1)
model.fit(X_train, y_train)
print(f"Accuracy на тесте: {accuracy_score(y_test, model.predict(X_test)):.4f}")

Accuracy на тесте: 0.8554


In [8]:
# Сохраняем baseline
from pickle import dump
with open("baseline_rf.pkl", "wb") as f:
    dump(model, f, protocol=5)

In [11]:
# Подберем параметры Random Forest с помощью grid search
num_trees = [25, 50, 75, 100, 125, 150, 175, 200]
max_depth = [None, 10, 40, 70]

model = RandomForestClassifier(n_jobs=-1)

best_model = GridSearchCV(
    model, 
    param_grid={
        'n_estimators': num_trees,
        'max_depth': max_depth
    }, 
    cv=3,
    verbose=4).fit(X_train, y_train)

print(f"Accuracy на тесте: {accuracy_score(y_test, best_model.predict(X_test)):.4f}")

Fitting 3 folds for each of 32 candidates, totalling 96 fits
[CV 1/3] END ...max_depth=None, n_estimators=25;, score=0.793 total time= 1.4min
[CV 2/3] END ...max_depth=None, n_estimators=25;, score=0.781 total time= 1.3min
[CV 3/3] END ...max_depth=None, n_estimators=25;, score=0.783 total time= 1.6min
[CV 1/3] END ...max_depth=None, n_estimators=50;, score=0.815 total time= 2.7min
[CV 2/3] END ...max_depth=None, n_estimators=50;, score=0.807 total time= 1.9min
[CV 3/3] END ...max_depth=None, n_estimators=50;, score=0.811 total time=  57.1s
[CV 1/3] END ...max_depth=None, n_estimators=75;, score=0.823 total time= 1.7min
[CV 2/3] END ...max_depth=None, n_estimators=75;, score=0.814 total time= 1.2min
[CV 3/3] END ...max_depth=None, n_estimators=75;, score=0.813 total time= 1.2min
[CV 1/3] END ..max_depth=None, n_estimators=100;, score=0.829 total time= 1.8min
[CV 2/3] END ..max_depth=None, n_estimators=100;, score=0.816 total time= 1.6min
[CV 3/3] END ..max_depth=None, n_estimators=100;

In [15]:
# Применяем QuantileTransformer к фичам
from sklearn.preprocessing import QuantileTransformer

transformer = QuantileTransformer(output_distribution='normal').fit(X_train)
quant_X_train = transformer.transform(X_train)
quant_X_test = transformer.transform(X_test)

quant_model = RandomForestClassifier(n_jobs=-1)
quant_model.fit(quant_X_train, y_train)

print(f"Accuracy на тесте c normal: {accuracy_score(y_test, quant_model.predict(quant_X_test)):.4f}")

del quant_X_train, quant_X_test


transformer = QuantileTransformer(output_distribution='uniform').fit(X_train)
quant_X_train = transformer.transform(X_train)
quant_X_test = transformer.transform(X_test)

quant_model = RandomForestClassifier(n_jobs=-1)
quant_model.fit(quant_X_train, y_train)

print(f"Accuracy на тесте c normal: {accuracy_score(y_test, quant_model.predict(quant_X_test)):.4f}")

del quant_X_train, quant_X_test

Accuracy на тесте c normal: 0.8501
Accuracy на тесте c normal: 0.8583


In [16]:
# Применяем PCA к фичам
from sklearn.decomposition import PCA

pca_model = PCA(n_components=1024).fit(X_train)
pca_X_train = pca_model.transform(X_train)
pca_X_test = pca_model.transform(X_test)

pca_model = RandomForestClassifier(n_jobs=-1)
pca_model.fit(pca_X_train, y_train)

print(f"Accuracy на тесте c normal: {accuracy_score(y_test, pca_model.predict(pca_X_test)):.4f}")

del pca_X_train, pca_X_test

Accuracy на тесте c normal: 0.8199


### Эксперименты с Random Forest на фичах других EfficientNet-ов моделей

In [17]:
# Загружаем данные
X_train = pd.read_csv("./processed_images_data/X_train_V1_B5_456.csv",  header=None)
y_train = pd.read_csv("./processed_images_data/y_train_V1_B5_456.csv", header=None).values.ravel()
X_test = pd.read_csv("./processed_images_data/X_test_V1_B5_456.csv",  header=None)
y_test = pd.read_csv("./processed_images_data/y_test_V1_B5_456.csv", header=None).values.ravel()

In [18]:
# Подберем параметры Random Forest с помощью grid search
num_trees = [25, 50, 75, 100, 125, 150, 175, 200]
max_depth = [None, 10, 40, 70]

model = RandomForestClassifier()

best_model = GridSearchCV(
    model, 
    param_grid={
        'n_estimators': num_trees,
        'max_depth': max_depth
    }, 
    cv=3,
    verbose=4).fit(X_train, y_train)

print(f"Accuracy на тесте: {accuracy_score(y_test, best_model.predict(X_test)):.4f}")

Fitting 3 folds for each of 32 candidates, totalling 96 fits
[CV 1/3] END ...max_depth=None, n_estimators=25;, score=0.791 total time= 1.4min
[CV 2/3] END ...max_depth=None, n_estimators=25;, score=0.782 total time= 1.4min
[CV 3/3] END ...max_depth=None, n_estimators=25;, score=0.792 total time= 1.4min
[CV 1/3] END ...max_depth=None, n_estimators=50;, score=0.813 total time= 2.6min
[CV 2/3] END ...max_depth=None, n_estimators=50;, score=0.806 total time= 2.6min
[CV 3/3] END ...max_depth=None, n_estimators=50;, score=0.806 total time= 2.6min
[CV 1/3] END ...max_depth=None, n_estimators=75;, score=0.826 total time= 3.9min
[CV 2/3] END ...max_depth=None, n_estimators=75;, score=0.815 total time= 3.8min
[CV 3/3] END ...max_depth=None, n_estimators=75;, score=0.816 total time= 3.8min
[CV 1/3] END ..max_depth=None, n_estimators=100;, score=0.826 total time= 4.9min
[CV 2/3] END ..max_depth=None, n_estimators=100;, score=0.816 total time= 5.0min
[CV 3/3] END ..max_depth=None, n_estimators=100;

In [19]:
# Загружаем данные
X_train = pd.read_csv("./processed_images_data/X_train_V2_L_480.csv",  header=None)
y_train = pd.read_csv("./processed_images_data/y_train_V2_L_480.csv", header=None).values.ravel()
X_test = pd.read_csv("./processed_images_data/X_test_V2_L_480.csv",  header=None)
y_test = pd.read_csv("./processed_images_data/y_test_V2_L_480.csv", header=None).values.ravel()

In [20]:
# Подберем параметры Random Forest с помощью grid search
num_trees = [25, 50, 75, 100, 125, 150, 175, 200]
max_depth = [None, 10, 40, 70]

model = RandomForestClassifier()

best_model = GridSearchCV(
    model, 
    param_grid={
        'n_estimators': num_trees,
        'max_depth': max_depth
    }, 
    cv=3,
    verbose=4).fit(X_train, y_train)

print(f"Accuracy на тесте: {accuracy_score(y_test, best_model.predict(X_test)):.4f}")

Fitting 3 folds for each of 32 candidates, totalling 96 fits
[CV 1/3] END ...max_depth=None, n_estimators=25;, score=0.770 total time=  42.9s
[CV 2/3] END ...max_depth=None, n_estimators=25;, score=0.755 total time=  42.5s
[CV 3/3] END ...max_depth=None, n_estimators=25;, score=0.764 total time=  43.3s
[CV 1/3] END ...max_depth=None, n_estimators=50;, score=0.781 total time= 1.4min
[CV 2/3] END ...max_depth=None, n_estimators=50;, score=0.789 total time= 1.4min
[CV 3/3] END ...max_depth=None, n_estimators=50;, score=0.786 total time= 1.4min
[CV 1/3] END ...max_depth=None, n_estimators=75;, score=0.799 total time= 2.1min
[CV 2/3] END ...max_depth=None, n_estimators=75;, score=0.795 total time= 2.1min
[CV 3/3] END ...max_depth=None, n_estimators=75;, score=0.791 total time= 2.1min
[CV 1/3] END ..max_depth=None, n_estimators=100;, score=0.802 total time= 2.6min
[CV 2/3] END ..max_depth=None, n_estimators=100;, score=0.802 total time= 2.3min
[CV 3/3] END ..max_depth=None, n_estimators=100;

In [21]:
# Загружаем данные
X_train = pd.read_csv("./processed_images_data/X_train_V2_M_480.csv",  header=None)
y_train = pd.read_csv("./processed_images_data/y_train_V2_M_480.csv", header=None).values.ravel()
X_test = pd.read_csv("./processed_images_data/X_test_V2_M_480.csv",  header=None)
y_test = pd.read_csv("./processed_images_data/y_test_V2_M_480.csv", header=None).values.ravel()

In [22]:
# Подберем параметры Random Forest с помощью grid search
num_trees = [25, 50, 75, 100, 125, 150, 175, 200]
max_depth = [None, 10, 40, 70]

model = RandomForestClassifier()

best_model = GridSearchCV(
    model, 
    param_grid={
        'n_estimators': num_trees,
        'max_depth': max_depth
    }, 
    cv=3,
    verbose=4).fit(X_train, y_train)

print(f"Accuracy на тесте: {accuracy_score(y_test, best_model.predict(X_test)):.4f}")

Fitting 3 folds for each of 32 candidates, totalling 96 fits
[CV 1/3] END ...max_depth=None, n_estimators=25;, score=0.771 total time=  34.6s
[CV 2/3] END ...max_depth=None, n_estimators=25;, score=0.764 total time=  36.1s
[CV 3/3] END ...max_depth=None, n_estimators=25;, score=0.768 total time=  35.0s
[CV 1/3] END ...max_depth=None, n_estimators=50;, score=0.791 total time= 1.2min
[CV 2/3] END ...max_depth=None, n_estimators=50;, score=0.797 total time= 1.2min
[CV 3/3] END ...max_depth=None, n_estimators=50;, score=0.786 total time= 1.2min
[CV 1/3] END ...max_depth=None, n_estimators=75;, score=0.799 total time= 1.8min
[CV 2/3] END ...max_depth=None, n_estimators=75;, score=0.806 total time= 1.8min
[CV 3/3] END ...max_depth=None, n_estimators=75;, score=0.800 total time= 1.7min
[CV 1/3] END ..max_depth=None, n_estimators=100;, score=0.801 total time= 2.4min
[CV 2/3] END ..max_depth=None, n_estimators=100;, score=0.810 total time= 2.3min
[CV 3/3] END ..max_depth=None, n_estimators=100;

In [23]:
# Загружаем данные
X_train = pd.read_csv("./processed_images_data/X_train_V2_S_384.csv",  header=None)
y_train = pd.read_csv("./processed_images_data/y_train_V2_S_384.csv", header=None).values.ravel()
X_test = pd.read_csv("./processed_images_data/X_test_V2_S_384.csv",  header=None)
y_test = pd.read_csv("./processed_images_data/y_test_V2_S_384.csv", header=None).values.ravel()

In [24]:
# Подберем параметры Random Forest с помощью grid search
num_trees = [25, 50, 75, 100, 125, 150, 175, 200]
max_depth = [None, 10, 40, 70]

model = RandomForestClassifier()

best_model = GridSearchCV(
    model, 
    param_grid={
        'n_estimators': num_trees,
        'max_depth': max_depth
    }, 
    cv=3,
    verbose=4).fit(X_train, y_train)

print(f"Accuracy на тесте: {accuracy_score(y_test, best_model.predict(X_test)):.4f}")

Fitting 3 folds for each of 32 candidates, totalling 96 fits
[CV 1/3] END ...max_depth=None, n_estimators=25;, score=0.724 total time=  35.6s
[CV 2/3] END ...max_depth=None, n_estimators=25;, score=0.725 total time=  35.7s
[CV 3/3] END ...max_depth=None, n_estimators=25;, score=0.728 total time=  35.6s
[CV 1/3] END ...max_depth=None, n_estimators=50;, score=0.745 total time= 1.2min
[CV 2/3] END ...max_depth=None, n_estimators=50;, score=0.752 total time= 1.2min
[CV 3/3] END ...max_depth=None, n_estimators=50;, score=0.751 total time= 1.2min
[CV 1/3] END ...max_depth=None, n_estimators=75;, score=0.756 total time= 1.8min
[CV 2/3] END ...max_depth=None, n_estimators=75;, score=0.760 total time= 1.8min
[CV 3/3] END ...max_depth=None, n_estimators=75;, score=0.753 total time= 1.7min
[CV 1/3] END ..max_depth=None, n_estimators=100;, score=0.762 total time= 2.4min
[CV 2/3] END ..max_depth=None, n_estimators=100;, score=0.759 total time= 2.3min
[CV 3/3] END ..max_depth=None, n_estimators=100;

In [25]:
# Загружаем данные
X_train = pd.read_csv("./processed_images_data/X_train_V2_S_450.csv",  header=None)
y_train = pd.read_csv("./processed_images_data/y_train_V2_S_450.csv", header=None).values.ravel()
X_test = pd.read_csv("./processed_images_data/X_test_V2_S_450.csv",  header=None)
y_test = pd.read_csv("./processed_images_data/y_test_V2_S_450.csv", header=None).values.ravel()

In [26]:
# Подберем параметры Random Forest с помощью grid search
num_trees = [25, 50, 75, 100, 125, 150, 175, 200]
max_depth = [None, 10, 40, 70]

model = RandomForestClassifier()

best_model = GridSearchCV(
    model, 
    param_grid={
        'n_estimators': num_trees,
        'max_depth': max_depth
    }, 
    cv=3,
    verbose=4).fit(X_train, y_train)

print(f"Accuracy на тесте: {accuracy_score(y_test, best_model.predict(X_test)):.4f}")

Fitting 3 folds for each of 32 candidates, totalling 96 fits
[CV 1/3] END ...max_depth=None, n_estimators=25;, score=0.725 total time=  35.0s
[CV 2/3] END ...max_depth=None, n_estimators=25;, score=0.741 total time=  34.8s
[CV 3/3] END ...max_depth=None, n_estimators=25;, score=0.727 total time=  35.8s
[CV 1/3] END ...max_depth=None, n_estimators=50;, score=0.759 total time= 1.2min
[CV 2/3] END ...max_depth=None, n_estimators=50;, score=0.749 total time= 1.2min
[CV 3/3] END ...max_depth=None, n_estimators=50;, score=0.746 total time= 1.2min
[CV 1/3] END ...max_depth=None, n_estimators=75;, score=0.765 total time= 1.8min
[CV 2/3] END ...max_depth=None, n_estimators=75;, score=0.764 total time= 2.0min
[CV 3/3] END ...max_depth=None, n_estimators=75;, score=0.759 total time= 1.8min
[CV 1/3] END ..max_depth=None, n_estimators=100;, score=0.764 total time= 2.6min
[CV 2/3] END ..max_depth=None, n_estimators=100;, score=0.770 total time= 2.3min
[CV 3/3] END ..max_depth=None, n_estimators=100;